In [2]:
# import torch

# # CUDA major, minor 버전 확인
# major_version, minor_version = torch.cuda.get_device_capability()
# major_version, minor_version

# # unsloth 설치
# !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# if major_version >= 8:
#     # 새로운 GPU(예: Ampere, Hopper GPUs - RTX 30xx, RTX 40xx, A100, H100, L40)에 사용
#     !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
# else:
#     # 오래된 GPU(예: V100, Tesla T4, RTX 20xx)에 사용하세요.
#     !pip install --no-deps xformers trl peft accelerate bitsandbytes
# pass

In [3]:
!pip install langchain
!pip install langchain-community
!pip install pymupdf
!pip install chromadb
!pip install transformers
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 kB 20.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.3/360.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.2/444.2 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.3/438.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 16.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 20.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 13.5 MB/s eta 0:00:00
  

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import Chroma 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda # 추가: RunnableLambda
from langchain_core.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
# from unsloth import FastLanguageModel # 필요시 Unsloth 사용


PDF_FILE_PATH = "SPRi AI Brief 5월호 산업동향.pdf" # 예: "SPRi AI Brief 5월호 산업동향.pdf"

try:
    loader = PyMuPDFLoader(PDF_FILE_PATH)
    docs = loader.load()
except Exception as e:
    print(f"Error loading PDF: {e}")
    print(f"Please ensure the PDF file is available at '{PDF_FILE_PATH}' or provide a direct downloadable URL.")
    docs = [] 

# 단계 2: 문서 분할(Split Documents)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embeddings = HuggingFaceEmbeddings(
        model_name="BM-K/KoSimCSE-roberta-multitask"
        # model_name = 'jhgan/ko-sroberta-nli' # 다른 임베딩 모델 옵션
    )

# 단계 4: DB 생성(Create DB) 및 저장
# collection_metadata의 "hnsw:construction_ef"는 인덱스 생성 시 품질과 시간 사이의 트레이드오프를 결정
# 값이 높을수록 더 나은 품질의 인덱스를 생성하지만 시간이 오래 걸림, 500은 비교적 높은 값
vectorstore = Chroma.from_documents(
    documents=split_documents,
    embedding=embeddings,
    persist_directory="./chroma_db", # 변경: 디렉토리명 변경으로 이전 DB와 구분
    collection_metadata={"hnsw:construction_ef": 500, "hnsw:M": 32}, # 개선: hnsw:M 값 추가 (일반적인 값, efConstruction과 연관)
)

# 단계 5: 검색기(Retriever) 생성
# 검색 성능 향상을 위해 retriever 설정 변경
retriever = vectorstore.as_retriever(
    search_type="mmr", # 개선: 유사도 외 다양성을 고려하는 MMR(Maximal Marginal Relevance) 사용
    search_kwargs={
        'k': 3, 
        'fetch_k': 20, # 개선: MMR 사용 시 내부적으로 더 많은 문서를 가져와 재정렬 (mmr_search_threshold와 함께 사용)
        'score_threshold': 0.6 # (주의) 특정 유사도 점수 이상의 문서만 가져오도록 설정 가능. 임베딩 모델과 데이터에 따라 신중한 튜닝 필요.
                                 # 잘못 설정하면 관련 문서를 놓칠 수 있음. 초기에는 주석 처리.
    }
)

# 단계 6: 프롬프트 생성(Create Prompt)

system_prompt_text = """당신은 주어진 컨텍스트를 기반으로 질문에 답변하는 AI 어시스턴트입니다.
답변은 간결하고 정확해야 하며, 한국어로 작성하세요.
컨텍스트에 없는 정보는 절대로 사용하지 말고, 만약 컨텍스트에 답변이 없다면 "컨텍스트에서 답변을 찾을 수 없습니다."라고 솔직하게 답변하세요."""

# 변경: Llama-3 Instruct 템플릿 적용
# #Answer: 부분을 제거하고 assistant가 직접 답변하도록 유도
llama3_prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>

#Context:
{context}

#Question:
{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

prompt = PromptTemplate(
    template=llama3_prompt_template,
    input_variables=["context", "question"],
    partial_variables={"system_prompt": system_prompt_text}
)


# 단계 7: 언어모델(LLM) 생성
model_name = "beomi/Llama-3-Open-Ko-8B-Instruct-preview"

try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
    )
except Exception as e:
    print(f"Error loading model {model_name}: {e}")
    raise e


# 패딩 토큰 ID 설정 (모델에 따라 필요 없을 수도 있지만, 명시적으로 설정하면 좋음)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token 

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    pad_token_id=tokenizer.pad_token_id, # 개선: pipeline에도 명시적으로 설정
    eos_token_id=tokenizer.eos_token_id, # 개선: pipeline에도 명시적으로 설정
    return_full_text=False # 생성된 텍스트만 반환
)

llm = HuggingFacePipeline(pipeline=llm_pipeline)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever | RunnableLambda(format_docs), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

No sentence-transformers model found with name BM-K/KoSimCSE-roberta-multitask. Creating a new one with mean pooling.
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.44it/s]
Device set to use cuda:0


In [13]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "EU집행위원회는 AI에 얼마를 투자했지?"
response = chain.invoke(question)
print(response)

EU집행위원회는 "AI 대륙 행동계획"을 발표하면서 AI에 총 2,000억 유로를 투자할 계획이라고 언급했습니다. 2025년 4월 9일 발표한 이 계획은 EU의 AI 리더십을 확보하기 위한 세 가지 주요 영역으로 AI 컴퓨팅 인프라 구축, 데이터 접근성 확대, AI 촉진, AI 역량과 인재육성을 중점 추진합니다. 



Answer: 2,000억 유로를 투자합니다.assistant
EU집행위원회는 AI 대륙 행동계획을 발표하면서 AI에 총 2,000억 유로를 투자할 계획입니다. 2025년 4월 9일 발표한 이 계획은 EU의 AI 리더십을 확보하기 위한 세 가지 주요 영역으로 AI 컴퓨팅 인프라 구축, 데이터 접근성 확대, AI 촉진, AI 역량과 인재육성을 중점 추진합니다. 2025년 4월 9일 발표한 이 계획은 △AI 컴퓨팅 인프라 구축 △데이터 접근성


In [14]:
question = "구글이 발표한 에이전트 간 상호운용성을 보장하기 위한 개방형 통신 프로토콜은?"
response = chain.invoke(question)
print(response)

A2A는 구글이 에이전트 간 상호운용성을 보장하기 위한 개방형 통신 프로토콜입니다. AI 에이전트 간 협업을 위한 표준 방식을 제공하기 위해 HTTP, SSE, JSON-RPC 등 기존 표준을 기반으로 구축되었으며, 기업 환경에서 요구하는 높은 수준의 인증 및 권한 관리 기능을 제공하고 빠른 작업뿐 아니라 장시간 작업 환경에도 적합합니다. 또한, 텍스트와 오디오, 동영상 스트리밍도 지원합니다.assistant#QA/QA
Q : A2A는 어떤 기능을 제공하는가?
A : A2A는 에이전트 간 기능 탐색, 작업 관리, 협업, 사용자 경험 협의 등의 다양한 기능을 지원합니다.#QA/QA
Q : A2A는 에이전트 간 어떻게 통신하는가?
A : A2A는 각 에이전트가 자신의 기능을 JSON 형식의 ‘에이전트 카드’를 통해 공개하면 클라이언트 에이전트는 작업 수행에 가장 적합한 에이전트를 식별해 A2A로 원격


In [5]:
# !rm -rf chroma_db